# ИД23-1 Маслов АН
## Парсер Википедии (разработан с помощью Cursor)
### Парсинг проходил по датам 14-19 ноября (qyy1114.xml - qyy1119.xml), парсились только новые люди и люди у которых не было года или професии в файлах pyy1114.xml - pyy1119.xml

In [16]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import re
import time
from urllib.parse import unquote, urljoin, urlparse
import html
import sys
import io

# Проверка на UTF-8 в Jupyter/IPython
try:
    if hasattr(sys.stdout, 'buffer') and sys.stdout.encoding != 'utf-8':
        sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
    if hasattr(sys.stderr, 'buffer') and sys.stderr.encoding != 'utf-8':
        sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8')
except (AttributeError, ValueError):
    pass

# Конфигурация
BASE_URL = "https://ru.wikipedia.org"
CATEGORY_URL = "https://ru.wikipedia.org/wiki/Категория:Родившиеся_8_ноября"
EXISTING_XML = "pyy1108.xml"
OUTPUT_XML = "qyy1108.xml"

# Перечень профессий на русском языке
OCCUPATIONS = [
    "писатель", "поэт", "прозаик", "драматург", "журналист", "публицист", "эссеист", "критик", "литературный критик",
    "библиограф", "библиофил", "лексикограф", "литературовед", "историк литературы",
    "художник", "скульптор", "архитектор", "дизайнер", "иллюстратор", "график", "живописец", "портретист",
    "карикатурист", "гравёр", "художник-гравёр", "художник-график", "художник-постановщик", "художник-мультипликатор",
    "деятель изобразительного искусства", "керамист", "керамистка", "гобеленистка", "рисовальщик", "рисовальщица",
    "актёр", "актриса", "режиссёр", "кинорежиссёр", "сценарист", "оператор", "кинематографист", "кинооператор",
    "театральный деятель", "танцор", "хореограф", "балетмейстер", "артистка балета", "театральный актёр",
    "театральный педагог", "театральный критик", "театровед", "кинокритик", "киновед", "искусствовед",
    "музыкант", "композитор", "певец", "дирижёр", "пианист", "пианистка", "басистка",
    "учёный", "исследователь", "профессор", "преподаватель", "учитель", "лектор", "преподаватель университета",
    "физик", "химик", "математик", "биолог", "ботаник", "геолог", "историк", "географ", "археолог", "лингвист",
    "антрополог", "этнограф", "палеонтолог", "палеоботаник", "вирусолог", "микробиолог", "биохимик", "физиолог",
    "анатом", "энтомолог", "зоолог", "орнитолог", "ихтиолог", "лепидоптеролог", "лихенолог", "селекционер",
    "филолог", "классицист", "классицистка", "романист", "нумизматка", "византолог",
    "врач", "медик", "хирург", "психиатр", "невролог", "патолог", "фармаколог", "военный врач",
    "инженер", "конструктор", "изобретатель", "авиационный инженер", "горный инженер", "военный инженер",
    "инженер-строитель", "инженер-механик", "инженер-гидротехник", "инженер-архитектор",
    "политик", "государственный деятель", "дипломат", "юрист", "адвокат", "практикующий юрист", "судья",
    "прокурор", "депутат", "посол", "министр", "президент", "премьер-министр",
    "военный деятель", "военнослужащий", "офицер", "полководец", "лётчик", "солдат", "капитан", "генерал",
    "адмирал", "командир", "разведчик", "мемуарист",
    "спортсмен", "футболист", "тренер", "тренер-преподаватель", "бейсболист", "хоккеист", "биатлонист",
    "бобслеист", "скелетонист", "автогонщик", "раллийный автогонщик",
    "предприниматель", "бизнесмен", "торговец", "банкир", "менеджер", "промышленник", "магнат",
    "священник", "религиозный деятель", "монах", "раввин", "настоятель", "миссионер", "теолог", "богослов",
    "философ", "философ науки", "историк науки", "культуролог", "социолог", "экономист", "политолог",
    "библиотекарь", "переводчик", "редактор", "издатель", "фотограф", "кинопродюсер", "телепродюсер",
    "телеведущий", "радиоведущий", "подкастер", "обозреватель", "корреспондент", "репортёр",
    "модель", "фотомодель", "порноактриса", "эротическая модель", "участница конкурса красоты",
    "блогер", "видеоблогер", "стример", "тиктокер", "ютубер",
    "модельер", "кутюрье", "переплётчица",
    "долгожитель", "аристократ", "аристократка", "фрейлина", "аэронавт", "капитан судна",
    "капитан речного флота", "шеф-повар", "агроном", "доярка", "свинарка", "шахтёр", "рабочий", "крестьянин",
    "партизан", "революционерка", "активист", "активистка", "гуманистка", "правозащитница",
    "коллекционер", "коллекционерка", "коллекционер искусства"
]

# Категории профессий и соответствующие им ключевые слова на русском языке
OCCUPATION_CATEGORIES = {
    "Писатель": ["писатель", "поэт", "прозаик", "драматург", "журналист", "публицист", "переводчик", "редактор", "издатель"],
    "Художник": ["художник", "скульптор", "архитектор", "дизайнер", "иллюстратор", "фотограф"],
    "Кинематографист": ["актёр", "актриса", "режиссёр", "кинорежиссёр", "сценарист", "оператор", "кинематографист"],
    "Театральный деятель": ["театральный деятель", "актёр", "актриса", "режиссёр", "танцор", "хореограф"],
    "Учёный": ["учёный", "исследователь", "профессор", "преподаватель", "учитель", "физик", "химик", "математик", "биолог", "ботаник", "геолог", "историк", "географ", "археолог", "лингвист"],
    "Врач": ["врач", "медик", "хирург", "психиатр", "невролог"],
    "Военный деятель": ["военный деятель", "военнослужащий", "офицер", "полководец", "лётчик"],
    "Государственный деятель": ["государственный деятель", "политик", "дипломат", "юрист", "адвокат"],
    "Политик": ["политик", "государственный деятель"],
    "Предприниматель": ["предприниматель", "бизнесмен", "торговец"],
    "Религиозный деятель": ["религиозный деятель", "священник", "монах"],
    "Философ": ["философ"],
    "Персона": ["долгожитель", "аристократ", "аристократка", "фрейлина"],
    "Порноактриса": ["порноактриса"],
    "Фотомодель": ["фотомодель", "модель"],
    "Модельер": ["модельер"],
    "Блогер": ["блогер", "видеоблогер", "стример", "тиктокер"],
    "Ректор": ["ректор"],
    "Священник": ["священник"],
    "Убийца": ["убийца"],
    "Футболист": ["футболист"],
    "Биатлонист": ["биатлонист"],
    "Карточка фотографа": ["фотограф"]
}

print("Конфигурация загружена.")


Конфигурация загружена.


In [17]:
def normalize_name(name):
    """Normalize a name to a standard format for comparison"""
    if not name:
        return ''
    # Декодировать URL-encoded строки
    normalized = unquote(name)
    # Заменить пробелы на подчеркивания для соответствия формату XML
    normalized = normalized.replace(' ', '_')
    return normalized

def load_existing_entries(xml_file):
    """Загрузить существующие записи из XML-файла с их данными для сравнения"""
    existing_data = {}  # Словарь: normalized_name -> {'year': year, 'occupation': occupation}
    existing_names = set()  # Множество всех вариантов имен для быстрого поиска
    
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for psn in root.findall('psn'):
            name = psn.get('h', '')
            if name:
                # Нормализовать имя
                normalized = normalize_name(name)
                
                # Извлечь год и профессию
                year = psn.get('y', '')
                occupation_attr = psn.get('p', '')
                
                # Извлечь профессию (удалить категорию, если присутствует, формат: "Категория;профессия")
                occupation = ''
                if occupation_attr:
                    if ';' in occupation_attr:
                        occupation = occupation_attr.split(';', 1)[1]  # Get part after semicolon
                    else:
                        occupation = occupation_attr
                
                # Сохранить данные в словарь
                existing_data[normalized] = {
                    'year': year if year and year != 'yyyy' else None,
                    'occupation': occupation
                }
                
                # Добавить все варианты имен в множество для быстрого поиска
                existing_names.add(normalized)
                existing_names.add(name)
                try:
                    from urllib.parse import quote
                    existing_names.add(quote(normalized, safe=''))
                    existing_names.add(quote(name, safe=''))
                except:
                    pass
        
        print(f"Загружено {len(existing_data)} существующих записей из {xml_file}")
        print(f"Пример записей: {list(existing_data.keys())[:3] if existing_data else 'None'}")
    except FileNotFoundError:
        print(f"Файл {xml_file} не найден, начинаем с чистого листа")
    except Exception as e:
        print(f"Ошибка загрузки {xml_file}: {e}")
    
    return existing_data, existing_names

existing_data, existing_names = load_existing_entries(EXISTING_XML)


Загружено 1314 существующих записей из pyy1119.xml
Пример записей: ['Абаджян,_Гаррий_Артушевич', 'Абдеррахим,_Муни', 'Абдулкагиров,_Магомед_Камингаджиевич']


In [18]:
def get_page_content(url):
    """Ищет содержимое страницы Википедии"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

def extract_year_of_birth(soup):
    """Вытаскивает год рождения из страницы Википедии"""
    # Пытаемся найти в инфобоксе (самый надежный)
    infobox = soup.find('table', class_='infobox')
    if infobox:
        # Ищет "Дата рождения" или "Родился/Родилась"
        for row in infobox.find_all('tr'):
            th = row.find('th')
            if th:
                th_text = th.get_text().strip()
                if 'Дата рождения' in th_text or 'Родился' in th_text or 'Родилась' in th_text:
                    td = row.find('td')
                    if td:
                        text = td.get_text()
                        # Ищем шаблоны даты с 4-значным годом (между 1000-2099)
                        years = re.findall(r'\b(1[0-9]{3}|20[0-2][0-9])\b', text)
                        if years:
                            # Возвращаем первый найденный год
                            return years[0]
    
    # Пытаемся найти в первом абзаце
    first_para = soup.find('div', class_='mw-parser-output')
    if first_para:
        paragraphs = first_para.find_all('p', recursive=False)
        for p in paragraphs[:3]:  # Проверяем первые 3 абзаца
            text = p.get_text()
            # Ищем шаблоны вроде "родился 14 ноября 1900 года" или "родился в 1900 году"
            patterns = [
                r'(?:родился|родилась)\s+(?:14\s+ноября\s+)?(\d{4})\s*(?:года|г\.|г)',
                r'(?:родился|родилась)\s+в\s+(\d{4})\s*(?:году|г\.|г)',
                r'(\d{4})\s*(?:года|г\.|г)\s*—\s*(?:родился|родилась)',
                r'14\s+ноября\s+(\d{4})',  # Прямой шаблон для "XX ноября YYYY"
            ]
            for pattern in patterns:
                year_match = re.search(pattern, text, re.IGNORECASE)
                if year_match:
                    year = year_match.group(1)
                    if 1000 <= int(year) <= 2099:
                        return year
            
            # Или просто найти 4-значный год рядом с "XX ноября" или "ноября"
            if '14 ноября' in text or 'ноября' in text:
                years = re.findall(r'\b(1[0-9]{3}|20[0-2][0-9])\b', text)
                if years:
                    return years[0]
    
    return None

def extract_occupation(soup):
    """Extract occupation from Wikipedia page"""
    occupations_found = []
    category_found = None
    
    # Пытаемся найти в инфобоксе (самый надежный)
    infobox = soup.find('table', class_='infobox')
    if infobox:
        # Ищет "Род деятельности" или "Профессия"
        for row in infobox.find_all('tr'):
            th = row.find('th')
            if th:
                th_text = th.get_text().strip()
                if 'Род деятельности' in th_text or 'Профессия' in th_text or 'Род занятий' in th_text:
                    td = row.find('td')
                    if td:
                        text = td.get_text().lower()
                        # Извлекаем профессии
                        for occ in OCCUPATIONS:
                            if occ.lower() in text and occ not in occupations_found:
                                occupations_found.append(occ)
    
    # Пытаемся найти в категориях (внизу страницы)
    categories = soup.find('div', id='catlinks')
    if categories:
        cat_text = categories.get_text().lower()
        # Проверяем каждую категорию
        for cat_name, occs in OCCUPATION_CATEGORIES.items():
            # Проверяем каждое ключевое слово в категории
            for occ in occs:
                if occ.lower() in cat_text:
                    if not category_found:  # Берем первую найденную категорию
                        category_found = cat_name
                    if occ not in occupations_found:
                        occupations_found.append(occ)
                    break
    
    # Пытаемся найти в первом абзаце, если не нашли ранее
    if not occupations_found:
        first_para = soup.find('div', class_='mw-parser-output')
        if first_para:
            paragraphs = first_para.find_all('p', recursive=False)
            for p in paragraphs[:3]:
                text = p.get_text().lower()
                for occ in OCCUPATIONS:
                    if occ.lower() in text and occ not in occupations_found:
                        occupations_found.append(occ)
    
    # Удаляем "персона" из профессий, если присутствует
    occupations_found = [occ for occ in occupations_found if occ != 'персона']
    
    # Если категория не найдена через категории страницы, определяем её по найденным профессиям
    if not category_found and occupations_found:
        # Ищем категорию по найденным профессиям
        for cat_name, occs in OCCUPATION_CATEGORIES.items():
            for occ in occupations_found:
                if occ in occs:
                    category_found = cat_name
                    break
            if category_found:
                break
    
    if occupations_found:
        # Возвращаем словарь с категорией и профессиями
        return {
            'category': category_found or 'Персона',
            'occupations': occupations_found[:5]  # Ограничиваем до 5 профессий
        }
    
    return None

print("Helper functions defined")


Helper functions defined


In [19]:
def parse_category_page(url):
    """Парсит страницу категории и возвращает список ссылок на людей и ссылку на следующую страницу"""
    soup = get_page_content(url)
    if not soup:
        return [], None
    
    person_links = []
    
    # Находит все ссылки на странице категории
    # Пытается разные возможные контейнеры
    content = soup.find('div', id='mw-pages')
    if not content:
        content = soup.find('div', class_='mw-category')
    if not content:
        content = soup.find('div', class_='mw-category-group')
    
    # Список префиксов специальных страниц для пропуска
    skip_prefixes = ['Категория:', 'Файл:', 'Шаблон:', 'Обсуждение:', 'Участник:', 'Портал:', 
                     'Википедия:', 'Служебная:', 'Медиа:', 'Справка:', 'Обсуждение_шаблона:',
                     'Обсуждение_участника:', 'Обсуждение_файла:', 'Обсуждение_категории:']
    
    if content:
        # Находит все ссылки в списке категории
        for link in content.find_all('a', href=True):
            href = link['href']
            if href.startswith('/wiki/'):
                title_part = href.split('/wiki/')[1].split('#')[0]
                # Декодируем для проверки, является ли это специальной страницей
                title_decoded = unquote(title_part)
                
                # Пропускаем специальные страницы
                should_skip = False
                for prefix in skip_prefixes:
                    if title_part.startswith(prefix) or title_decoded.startswith(prefix):
                        should_skip = True
                        break
                
                if not should_skip:
                    full_url = urljoin(BASE_URL, href)
                    # Используем URL-кодированный заголовок для согласованности с форматом XML  
                    person_links.append((title_part, full_url))
    
    # Удаляем дубликаты, сохраняя порядок
    seen = set()
    unique_links = []
    for title, url in person_links:
        if title not in seen:
            seen.add(title)
            unique_links.append((title, url))
    
    # Находит ссылку "Следующая страница" - улучшенный поиск
    next_link = None
    
    # Метод 1: Ищет ссылку с точным текстом "Следующая страница"
    for link in soup.find_all('a', href=True):
        link_text = link.get_text().strip()
        if link_text == 'Следующая страница':
            next_link = urljoin(BASE_URL, link['href'])
            break
    
    # Метод 2: Ищет в навигационных блоках (более комплексный поиск)
    if not next_link:
        # Пытается разные навигационные контейнеры
        nav_selectors = [
            ('div', {'class': 'mw-category-navigation'}),
            ('div', {'id': 'mw-pages'}),
            ('div', {'class': 'mw-category'}),
        ]
        
        for tag, attrs in nav_selectors:
            nav_divs = soup.find_all(tag, attrs)
            for nav_div in nav_divs:
                for link in nav_div.find_all('a', href=True):
                    link_text = link.get_text().strip()
                    # Проверяет наличие "Следующая страница" или вариаций
                    if 'Следующая' in link_text or 'следующая' in link_text or 'Следующая страница' in link_text:
                        href = link.get('href')
                        if href and href.startswith('/wiki/'):
                            next_link = urljoin(BASE_URL, href)
                            break
                if next_link:
                    break
            if next_link:
                break
    
    # Метод 3: Ищет ссылки с параметром "from" (пагинация)  
    if not next_link:
        # Страницы категорий Wikipedia используют параметр "from" для пагинации
        for link in soup.find_all('a', href=True):
            href = link.get('href', '')
            if 'from=' in href and 'Категория:Родившиеся_14_ноября' in href:
                next_link = urljoin(BASE_URL, href)
                break
    
    # Метод 4: Ищет класс "next" или подобный
    if not next_link:
        next_links = soup.find_all('a', class_=re.compile('next', re.I))
        for link in next_links:
            if link.get('href'):
                next_link = urljoin(BASE_URL, link['href'])
                break
    
    return unique_links, next_link

print("Category parser function defined")


Category parser function defined


In [20]:
def parse_person_page(url, name):
    """Parse individual person page and extract data"""
    soup = get_page_content(url)
    if not soup:
        return None
    
    # Extract year of birth
    year = extract_year_of_birth(soup)
    
    # Extract occupation (no category, just occupation)
    occupation = extract_occupation(soup)
    
    # Normalize name to match XML format (decoded, with underscores)
    name_normalized = normalize_name(name)
    
    return {
        'year': year,
        'name': name_normalized,  # Normalized format to match XML
        'occupation': occupation or ''  # Just occupation, no category
    }

print("Person parser function defined")


Person parser function defined


In [21]:
def write_xml(persons, output_file):
    """Записываем информацию о людях в XML-файл"""
    root = ET.Element('persons')
    root.set('day', '14-11')
    root.set('comm', f'Parsed {len(persons)} persons')
    
    # Сортировка по году, потом по имени
    def sort_key(person):
        year = person.get('year')
        name = person.get('name', '')
        # Конвертируем год в строку для правильной сортировки
        year_str = str(year) if year is not None else '9999'
        return (year_str, name)
    
    sorted_persons = sorted(persons, key=sort_key)
    
    for person in sorted_persons:
        psn = ET.SubElement(root, 'psn')
        if person.get('year'):
            psn.set('y', str(person['year']))
        else:
            psn.set('y', 'yyyy')  # Плейсхолдер если год не был найден
        
        psn.set('h', person['name'])
        
        # добавить профессию, если есть
        if person.get('occupation'):
            psn.set('p', person['occupation'])
    
    # Запись в файл с правильной кодировкой UTF-8
    tree = ET.ElementTree(root)
    ET.indent(tree, space='  ')
    # Запись файла с декларацией XML
    with open(output_file, 'wb') as f:
        f.write('<?xml version="1.0" encoding="utf-8"?>\n'.encode('utf-8'))
        tree.write(f, encoding='utf-8', xml_declaration=False)
    print(f"Written {len(persons)} persons to {output_file}")

print("XML writer function defined")


XML writer function defined


In [22]:
# Основная функция
def main():
    all_persons = []
    current_url = CATEGORY_URL
    page_count = 0
    total_processed = 0
    total_skipped_identical = 0
    total_updated = 0
    total_new = 0
    total_failed = 0
    
    print(f"Начинаем парсинг категории: {CATEGORY_URL}")
    print(f"Загружено существующих записей: {len(existing_data)}")
    
    while current_url:
        page_count += 1
        print(f"\n{'='*60}")
        print(f"--- Парсинг страницы {page_count} ---")
        print(f"URL: {current_url}")
        print(f"{'='*60}")
        
        # Парсинг страницы категории
        person_links, next_link = parse_category_page(current_url)
        print(f"Найдено {len(person_links)} ссылок на людей на этой странице")
        
        if not person_links:
            print("Ссылки на людей не найдены, пытаемся продолжить...")
        
        # Проходим по каждой персоне
        for idx, (name, url) in enumerate(person_links, 1):
            # Нормализуем имя для проверки
            name_normalized = normalize_name(name)
            name_display = unquote(name).replace('_', ' ')  # для отображения
            
            print(f"[{idx}/{len(person_links)}] Обработка: {name_display}")
            person_data = parse_person_page(url, name)
            
            if not person_data:
                total_failed += 1
                print(f"  ✗ Не удалось извлечь данные")
                time.sleep(0.5)
                continue
            
            # Проверяем, существует ли уже такая персона
            if name_normalized in existing_data:
                existing = existing_data[name_normalized]
                new_year = person_data.get('year')
                new_occupation_data = person_data.get('occupation_data')
                existing_year = existing.get('year')
                existing_occupation_data = existing.get('occupation_data')
                
                # Если не нашли другой год, используем существующий
                if new_year is None:
                    person_data['year'] = existing_year
                    new_year = existing_year
                
                # Нормализуем годы для сравнения (обрабатываем None и 'yyyy')
                new_year_str = str(new_year) if new_year else None
                existing_year_str = str(existing_year) if existing_year else None
                
                # Проверяем, отличаются ли данные
                year_different = new_year_str != existing_year_str
                
                # Меняем данные, только если у оригинала нет профессии
                # Проверяем, пуста ли профессия у существующей записи
                existing_occ_empty = not existing_occupation_data or not existing_occupation_data.get('occupations')
                
                # Меняем профессию только если у существующей записи она пуста
                if existing_occ_empty and new_occupation_data and new_occupation_data.get('occupations'):
                    occupation_different = True
                    # Используем новую профессию
                    person_data['occupation_data'] = new_occupation_data
                else:
                    # Сохраняем существующую профессию (не меняем)
                    occupation_different = False
                    person_data['occupation_data'] = existing_occupation_data
                
                # Обновляем, если год отличается ИЛИ профессия была обновлена
                if year_different or occupation_different:
                    # Данные отличаются, добавляем в вывод (перезапишем)
                    all_persons.append(person_data)
                    total_updated += 1
                    changes = []
                    if year_different:
                        changes.append(f"год: {existing_year_str} → {new_year_str}")
                    if occupation_different:
                        existing_occ_str = ''
                        new_occ_str = ''
                        if existing_occupation_data:
                            existing_occ_str = f"{existing_occupation_data.get('category', '')}; {', '.join(existing_occupation_data.get('occupations', []))}"
                        if new_occupation_data:
                            new_occ_str = f"{new_occupation_data.get('category', '')}; {', '.join(new_occupation_data.get('occupations', []))}"
                        changes.append(f"профессия: '{existing_occ_str}' → '{new_occ_str}'")
                    print(f"  ↻ Обновление: {', '.join(changes)}")
                else:
                    # Данные идентичны, пропускаем
                    total_skipped_identical += 1
                    print(f"  ✓ Данные идентичны, пропускаем")
            else:
                # Новая персона, добавляем
                all_persons.append(person_data)
                total_new += 1
                year = person_data.get('year', 'N/A')
                occupation_data = person_data.get('occupation_data')
                if occupation_data:
                    occ_str = f"{occupation_data.get('category', '')}; {', '.join(occupation_data.get('occupations', []))}"
                else:
                    occ_str = 'N/A'
                print(f"  + Новая: Год: {year}, Профессия: {occ_str}")
            
            total_processed += 1
            
            # Чтобы не забанило
            time.sleep(0.5)
        
        # Переход к следующей странице 
        current_url = next_link
        if next_link:
            print(f"\n→ Переход к следующей странице: {next_link}")
        else:
            print(f"\n→ Следующих страниц не найдено, завершаем парсинг.")
            break
    
    print(f"\n{'='*60}")
    print(f"=== Парсинг завершен ===")
    print(f"Всего страниц запарсено: {page_count}")
    print(f"Всего персон обработано: {total_processed}")
    print(f"Всего новых персон: {total_new}")
    print(f"Всего обновлено (отличающиеся данные): {total_updated}")
    print(f"Всего пропущено (идентичные данные): {total_skipped_identical}")
    print(f"Всего не удалось извлечь: {total_failed}")
    print(f"{'='*60}")
    
    # Запись в XML
    if all_persons:
        write_xml(all_persons, OUTPUT_XML)
        print(f"\n✓ Успешно записано {len(all_persons)} персон в {OUTPUT_XML}")
    else:
        print("\n⚠ Нет персон для записи (все были идентичны существующим данным)")

# Запуск парсера
if __name__ == "__main__":
    main()


Начинаем парсинг категории: https://ru.wikipedia.org/wiki/Категория:Родившиеся_19_ноября
Загружено существующих записей: 1314

--- Парсинг страницы 1 ---
URL: https://ru.wikipedia.org/wiki/Категория:Родившиеся_19_ноября
Error fetching https://ru.wikipedia.org/wiki/Категория:Родившиеся_19_ноября: 403 Client Error: Too many requests. Please respect our robot policy https://w.wiki/4wJS. (dd12474) for url: https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A0%D0%BE%D0%B4%D0%B8%D0%B2%D1%88%D0%B8%D0%B5%D1%81%D1%8F_19_%D0%BD%D0%BE%D1%8F%D0%B1%D1%80%D1%8F
Найдено 0 ссылок на людей на этой странице
Ссылки на людей не найдены, пытаемся продолжить...

→ Следующих страниц не найдено, завершаем парсинг.

=== Парсинг завершен ===
Всего страниц запарсено: 1
Всего персон обработано: 0
Всего новых персон: 0
Всего обновлено (отличающиеся данные): 0
Всего пропущено (идентичные данные): 0
Всего не удалось извлечь: 0

⚠ Нет персон для записи (все были идентичны существ